In [1]:
# ============================================================
# ANÁLISE EXPLORATÓRIA DETALHADA - IOWA LIQUOR SALES
# ============================================================

# Célula 1: Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


In [2]:
# ============================================================
# Célula 2: Carregar dados processados
# ============================================================

df = pd.read_csv('../data/processed/sales_data_processed.csv')
df['data_venda'] = pd.to_datetime(df['data_venda'])

print("📊 VISÃO GERAL DOS DADOS")
print("=" * 60)
print(f"Total de registros: {len(df):,}")
print(f"Período: {df['data_venda'].min()} até {df['data_venda'].max()}")
print(f"Colunas: {df.shape[1]}")
print("\n")
print(df.info())

📊 VISÃO GERAL DOS DADOS
Total de registros: 47,956
Período: 2025-12-26 00:00:00 até 2025-12-31 00:00:00
Colunas: 23


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47956 entries, 0 to 47955
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   data_venda            47956 non-null  datetime64[ns]
 1   loja                  47956 non-null  object        
 2   cidade                47941 non-null  object        
 3   regiao                47941 non-null  object        
 4   categoria             47956 non-null  object        
 5   produto               47956 non-null  object        
 6   fornecedor            47956 non-null  object        
 7   quantidade            47956 non-null  int64         
 8   custo_unitario        47956 non-null  float64       
 9   preco_unitario        47956 non-null  float64       
 10  receita               47956 non-null  float64       
 11  lucro         

In [12]:
# ============================================================
# Célula 3: ANÁLISE POR REGIÃO/ESTADO
# ============================================================

print("\n" + "=" * 60)
print("🌍 ANÁLISE POR REGIÃO")
print("=" * 60)

# Top 10 regiões por receita
top_regioes = df.groupby('regiao').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count',
    'margem_percentual': 'mean'
}).round(2)

top_regioes.columns = ['Receita Total', 'Lucro Total', 'Qtd Vendida', 'Num Transações', 'Margem Média %']
top_regioes = top_regioes.sort_values('Receita Total', ascending=False)

print("\n🏆 TOP 10 REGIÕES POR RECEITA:")
print(top_regioes.head(10))

print("\n📉 BOTTOM 10 REGIÕES POR RECEITA:")
print(top_regioes.tail(10))

# Visualização
fig = px.bar(
    top_regioes.head(15).reset_index(),
    x='regiao',
    y='Receita Total',
    title='Top 15 Regiões por Receita Total',
    labels={'regiao': 'Região', 'Receita Total': 'Receita (R$)'},
    color='Receita Total',
    color_continuous_scale='Viridis'
)
fig.show()


🌍 ANÁLISE POR REGIÃO

🏆 TOP 10 REGIÕES POR RECEITA:
               Receita Total  Lucro Total  Qtd Vendida  Num Transações  \
regiao                                                                   
POLK              1804497.56    601643.61       119839            9257   
LINN               714436.84    238312.80        50607            4396   
JOHNSON            531532.22    177111.83        29802            1978   
SCOTT              524509.58    175180.34        36734            2981   
DALLAS             386583.18    128610.70        17577            1216   
BLACK HAWK         345840.86    115294.54        25866            2536   
POTTAWATTAMIE      319252.87    106465.45        22744            1430   
WOODBURY           243390.77     81162.61        16497            1405   
CLINTON            234576.18     78180.72        14904            1025   
DUBUQUE            198025.63     65975.56        14770            1551   

               Margem Média %  
regiao                    

In [4]:
# ============================================================
# Célula 4: ANÁLISE POR CIDADE
# ============================================================

print("\n" + "=" * 60)
print("🏙️ ANÁLISE POR CIDADE")
print("=" * 60)

# Top cidades por receita
top_cidades = df.groupby('cidade').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count',
    'margem_percentual': 'mean',
    'loja': 'nunique'
}).round(2)

top_cidades.columns = ['Receita Total', 'Lucro Total', 'Qtd Vendida', 'Num Transações', 'Margem Média %', 'Num Lojas']
top_cidades = top_cidades.sort_values('Receita Total', ascending=False)

print("\n🏆 TOP 10 CIDADES POR RECEITA:")
print(top_cidades.head(10))

print("\n📉 BOTTOM 10 CIDADES POR RECEITA:")
print(top_cidades.tail(10))

# Ticket médio por cidade
top_cidades['Ticket Médio'] = top_cidades['Receita Total'] / top_cidades['Num Transações']
print("\n💰 TOP 10 CIDADES POR TICKET MÉDIO:")
print(top_cidades.nlargest(10, 'Ticket Médio')[['Receita Total', 'Ticket Médio', 'Num Transações']])

# Visualização
fig = px.treemap(
    top_cidades.head(20).reset_index(),
    path=['cidade'],
    values='Receita Total',
    color='Margem Média %',
    title='Top 20 Cidades - Tamanho = Receita, Cor = Margem',
    color_continuous_scale='RdYlGn'
)
fig.show()


🏙️ ANÁLISE POR CIDADE

🏆 TOP 10 CIDADES POR RECEITA:
                 Receita Total  Lucro Total  Qtd Vendida  Num Transações  \
cidade                                                                     
DES MOINES           925492.87    308485.82        61567            3837   
CEDAR RAPIDS         557386.75    185922.06        39512            3540   
WEST DES MOINES      459784.21    153021.02        24258            1864   
DAVENPORT            398175.22    133059.54        28667            2123   
COUNCIL BLUFFS       297437.91     99128.00        21450            1282   
ANKENY               265292.99     88567.96        15457            1087   
IOWA CITY            238370.43     79389.01        15638            1064   
CORALVILLE           234679.65     78249.39        10344             583   
SIOUX CITY           232126.07     77406.41        15342            1293   
WATERLOO             189744.78     63260.23        16252            1369   

                 Margem Média %  

In [5]:
# ============================================================
# Célula 5: ANÁLISE POR PRODUTO
# ============================================================

print("\n" + "=" * 60)
print("📦 ANÁLISE POR PRODUTO")
print("=" * 60)

# Top produtos por receita
top_produtos = df.groupby('produto').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count',
    'margem_percentual': 'mean',
    'preco_unitario': 'mean'
}).round(2)

top_produtos.columns = ['Receita Total', 'Lucro Total', 'Qtd Vendida', 'Num Transações', 'Margem Média %', 'Preço Médio']
top_produtos = top_produtos.sort_values('Receita Total', ascending=False)

print("\n🏆 TOP 20 PRODUTOS POR RECEITA:")
print(top_produtos.head(20))

print("\n📉 BOTTOM 20 PRODUTOS POR RECEITA:")
print(top_produtos.tail(20))

print("\n🔥 TOP 20 PRODUTOS POR QUANTIDADE VENDIDA:")
print(top_produtos.nlargest(20, 'Qtd Vendida')[['Qtd Vendida', 'Receita Total', 'Preço Médio']])

print("\n💎 TOP 20 PRODUTOS POR MARGEM DE LUCRO:")
print(top_produtos.nlargest(20, 'Margem Média %')[['Margem Média %', 'Receita Total', 'Lucro Total']])

# Visualização - Top 15 produtos
fig = go.Figure()

top_15_produtos = top_produtos.head(15).reset_index()

fig.add_trace(go.Bar(
    name='Receita',
    x=top_15_produtos['produto'],
    y=top_15_produtos['Receita Total'],
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    name='Lucro',
    x=top_15_produtos['produto'],
    y=top_15_produtos['Lucro Total'],
    marker_color='darkblue'
))

fig.update_layout(
    title='Top 15 Produtos - Receita vs Lucro',
    xaxis_title='Produto',
    yaxis_title='Valor (R$)',
    barmode='group',
    xaxis_tickangle=-45,
    height=500
)

fig.show()



📦 ANÁLISE POR PRODUTO

🏆 TOP 20 PRODUTOS POR RECEITA:
                                       Receita Total  Lucro Total  \
produto                                                             
TITOS HANDMADE VODKA                       378836.72    126306.01   
CROWN ROYAL REGAL APPLE                    249321.13     83140.00   
CROWN ROYAL                                232043.20     77375.25   
BLACK VELVET                               227394.65     75813.90   
JACK DANIELS OLD #7 BLACK LABEL            175216.42     58425.20   
FIREBALL CINNAMON WHISKEY                  142728.75     47589.52   
CAPTAIN MORGAN ORIGINAL SPICED             110136.99     36727.93   
JAMESON                                    105579.80     35201.03   
FIREBALL CINNAMON WHISKEY MINI SLEEVE       98415.00     32805.00   
CROWN ROYAL PEACH                           93929.31     31321.50   
CROWN ROYAL BLACKBERRY                      82768.90     27601.00   
BAILEYS ORIGINAL IRISH CREAM                8230

In [6]:
# ============================================================
# Célula 6: ANÁLISE POR CATEGORIA
# ============================================================

print("\n" + "=" * 60)
print("🏷️ ANÁLISE POR CATEGORIA")
print("=" * 60)

# Análise por categoria
categorias = df.groupby('categoria').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count',
    'margem_percentual': 'mean',
    'produto': 'nunique'
}).round(2)

categorias.columns = ['Receita Total', 'Lucro Total', 'Qtd Vendida', 'Num Transações', 'Margem Média %', 'Num Produtos']
categorias = categorias.sort_values('Receita Total', ascending=False)

print("\n📊 TODAS AS CATEGORIAS:")
print(categorias)

# Participação percentual
categorias['% Receita'] = (categorias['Receita Total'] / categorias['Receita Total'].sum() * 100).round(2)
print("\n📊 CATEGORIAS POR PARTICIPAÇÃO:")
print(categorias[['Receita Total', '% Receita', 'Margem Média %']].head(10))

# Visualização
fig = px.pie(
    categorias.head(10).reset_index(),
    values='Receita Total',
    names='categoria',
    title='Top 10 Categorias - Distribuição de Receita',
    hole=0.4
)
fig.show()


🏷️ ANÁLISE POR CATEGORIA

📊 TODAS AS CATEGORIAS:
                                      Receita Total  Lucro Total  Qtd Vendida  \
categoria                                                                       
CANADIAN WHISKIES                        1074050.01    358090.04        59831   
AMERICAN VODKAS                           860858.06    287039.11        93236   
STRAIGHT BOURBON WHISKIES                 730040.86    243268.51        30185   
100% AGAVE TEQUILA                        643790.71    214838.55        22802   
WHISKEY LIQUEUR                           433938.54    144671.30        73145   
SPICED RUM                                314891.15    104996.25        21846   
TEMPORARY & SPECIALTY PACKAGES            314516.01    104868.55        11316   
TENNESSEE WHISKIES                        308834.54    102976.52        13854   
IMPORTED CORDIALS & LIQUEURS              286354.55     95520.48        11300   
CREAM LIQUEURS                            245559.09     811

In [7]:
# ============================================================
# Célula 7: ANÁLISE POR FORNECEDOR/VENDEDOR
# ============================================================

print("\n" + "=" * 60)
print("👥 ANÁLISE POR FORNECEDOR")
print("=" * 60)

# Top fornecedores
top_fornecedores = df.groupby('fornecedor').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count',
    'margem_percentual': 'mean',
    'produto': 'nunique'
}).round(2)

top_fornecedores.columns = ['Receita Total', 'Lucro Total', 'Qtd Vendida', 'Num Transações', 'Margem Média %', 'Num Produtos']
top_fornecedores = top_fornecedores.sort_values('Receita Total', ascending=False)

print("\n🏆 TOP 15 FORNECEDORES POR RECEITA:")
print(top_fornecedores.head(15))

print("\n📉 BOTTOM 15 FORNECEDORES POR RECEITA:")
print(top_fornecedores.tail(15))

# Visualização
fig = px.bar(
    top_fornecedores.head(15).reset_index(),
    x='Receita Total',
    y='fornecedor',
    orientation='h',
    title='Top 15 Fornecedores por Receita',
    labels={'fornecedor': 'Fornecedor', 'Receita Total': 'Receita (R$)'},
    color='Margem Média %',
    color_continuous_scale='Viridis'
)
fig.update_layout(height=600)
fig.show()


👥 ANÁLISE POR FORNECEDOR

🏆 TOP 15 FORNECEDORES POR RECEITA:
                                          Receita Total  Lucro Total  \
fornecedor                                                             
DIAGEO AMERICAS                              1883670.26    628132.12   
SAZERAC COMPANY  INC                         1168906.80    389942.66   
JIM BEAM BRANDS                               544352.31    179617.87   
HEAVEN HILL BRANDS                            524655.98    174903.21   
PERNOD RICARD USA                             496111.71    165441.78   
BROWN FORMAN CORP.                            403568.83    134558.43   
FIFTH GENERATION DISTILLED SPIRITS, INC.      395806.07    131962.46   
BACARDI USA INC                               362957.58    121012.89   
PROXIMO                                       252917.66     84323.91   
LUXCO INC                                     227651.39     76221.72   
E & J GALLO WINERY                            170730.12     56844.90   
CA

In [8]:
# ============================================================
# Célula 8: ANÁLISE POR LOJA
# ============================================================

print("\n" + "=" * 60)
print("🏪 ANÁLISE POR LOJA")
print("=" * 60)

# Top lojas
top_lojas = df.groupby('loja').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count',
    'margem_percentual': 'mean',
    'cidade': 'first'
}).round(2)

top_lojas.columns = ['Receita Total', 'Lucro Total', 'Qtd Vendida', 'Num Transações', 'Margem Média %', 'Cidade']
top_lojas = top_lojas.sort_values('Receita Total', ascending=False)

print("\n🏆 TOP 20 LOJAS POR RECEITA:")
print(top_lojas.head(20))

print("\n📉 BOTTOM 20 LOJAS POR RECEITA:")
print(top_lojas.tail(20))



🏪 ANÁLISE POR LOJA

🏆 TOP 20 LOJAS POR RECEITA:
                                                 Receita Total  Lucro Total  \
loja                                                                          
CENTRAL CITY 2                                       258783.54     86274.45   
HY-VEE #3 / BDI / DES MOINES                         186284.11     62116.14   
WALL TO WALL WINE AND SPIRITS / WEST DES MOINES      140116.96     46728.80   
HY-VEE WINE AND SPIRITS #1 (1281) / IOWA CITY        128274.29     42716.78   
I-80 LIQUOR / COUNCIL BLUFFS                         121066.74     40321.92   
COSTCO WHOLESALE #1111 / CORALVILLE                  120353.76     40122.54   
BENZ DISTRIBUTING                                    112502.90     37494.58   
ANOTHER ROUND / DEWITT                               112171.86     37348.26   
COSTCO WHOLESALE #788 / WDM                          108924.12     35970.46   
MARSHALL BEER WINE SPIRITS                            68255.59     22771.26   
COS

In [9]:
# ============================================================
# Célula 9: ANÁLISE TEMPORAL
# ============================================================

print("\n" + "=" * 60)
print("📅 ANÁLISE TEMPORAL")
print("=" * 60)

# Vendas por mês
vendas_mes = df.groupby('nome_mes').agg({
    'receita': 'sum',
    'lucro': 'sum',
    'quantidade': 'sum',
    'data_venda': 'count'
}).round(2)

vendas_mes.columns = ['Receita', 'Lucro', 'Quantidade', 'Transações']
print("\n📊 VENDAS POR MÊS:")
print(vendas_mes)

# Vendas por dia da semana
vendas_dia_semana = df.groupby('nome_dia_semana').agg({
    'receita': 'sum',
    'data_venda': 'count'
}).round(2)

vendas_dia_semana.columns = ['Receita Total', 'Num Transações']
vendas_dia_semana['Receita Média'] = (vendas_dia_semana['Receita Total'] / vendas_dia_semana['Num Transações']).round(2)

print("\n📊 VENDAS POR DIA DA SEMANA:")
print(vendas_dia_semana)

# Visualização temporal
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=vendas_mes.index,
    y=vendas_mes['Receita'],
    mode='lines+markers',
    name='Receita',
    line=dict(color='blue', width=3)
))

fig.add_trace(go.Scatter(
    x=vendas_mes.index,
    y=vendas_mes['Lucro'],
    mode='lines+markers',
    name='Lucro',
    line=dict(color='green', width=3)
))

fig.update_layout(
    title='Evolução Temporal - Receita e Lucro',
    xaxis_title='Mês',
    yaxis_title='Valor (R$)',
    hovermode='x unified',
    height=500
)

fig.show()


📅 ANÁLISE TEMPORAL

📊 VENDAS POR MÊS:
            Receita      Lucro  Quantidade  Transações
nome_mes                                              
2025-12  8054904.90 2685476.52      537637       47956

📊 VENDAS POR DIA DA SEMANA:
                 Receita Total  Num Transações  Receita Média
nome_dia_semana                                              
Domingo              462956.11            3352         138.11
Quarta              1604043.73            9439         169.94
Segunda             2038192.55           11090         183.79
Sexta               1750428.43           10137         172.68
Sábado               154554.92            1285         120.28
Terça               2044729.16           12653         161.60


In [10]:
# ============================================================
# Célula 10: ANÁLISE DE CORRELAÇÃO E INSIGHTS
# ============================================================

print("\n" + "=" * 60)
print("🔍 INSIGHTS PRINCIPAIS")
print("=" * 60)

# Correlação entre variáveis numéricas
numeric_cols = ['quantidade', 'custo_unitario', 'preco_unitario', 'receita', 'lucro', 'margem_percentual']
correlacao = df[numeric_cols].corr().round(2)

print("\n📊 MATRIZ DE CORRELAÇÃO:")
print(correlacao)

# Visualização de correlação
fig = px.imshow(
    correlacao,
    text_auto=True,
    aspect="auto",
    title='Matriz de Correlação',
    color_continuous_scale='RdBu_r'
)
fig.show()


🔍 INSIGHTS PRINCIPAIS

📊 MATRIZ DE CORRELAÇÃO:
                   quantidade  custo_unitario  preco_unitario  receita  lucro  \
quantidade               1.00           -0.07           -0.07     0.76   0.76   
custo_unitario          -0.07            1.00            1.00     0.11   0.11   
preco_unitario          -0.07            1.00            1.00     0.11   0.11   
receita                  0.76            0.11            0.11     1.00   1.00   
lucro                    0.76            0.11            0.11     1.00   1.00   
margem_percentual       -0.01            0.04            0.04     0.01   0.01   

                   margem_percentual  
quantidade                     -0.01  
custo_unitario                  0.04  
preco_unitario                  0.04  
receita                         0.01  
lucro                           0.01  
margem_percentual               1.00  


In [11]:
# ============================================================
# RESUMO EXECUTIVO
# ============================================================

print("\n" + "=" * 80)
print("📋 RESUMO EXECUTIVO - PRINCIPAIS INSIGHTS")
print("=" * 80)

print(f"\n🌍 MELHOR REGIÃO: {top_regioes.index[0]}")
print(f"   - Receita: R$ {top_regioes.iloc[0]['Receita Total']:,.2f}")
print(f"   - Margem: {top_regioes.iloc[0]['Margem Média %']:.2f}%")

print(f"\n🌍 PIOR REGIÃO: {top_regioes.index[-1]}")
print(f"   - Receita: R$ {top_regioes.iloc[-1]['Receita Total']:,.2f}")

print(f"\n🏙️ MELHOR CIDADE: {top_cidades.index[0]}")
print(f"   - Receita: R$ {top_cidades.iloc[0]['Receita Total']:,.2f}")
print(f"   - Número de Lojas: {int(top_cidades.iloc[0]['Num Lojas'])}")

print(f"\n🏙️ PIOR CIDADE: {top_cidades.index[-1]}")
print(f"   - Receita: R$ {top_cidades.iloc[-1]['Receita Total']:,.2f}")

print(f"\n📦 PRODUTO MAIS VENDIDO: {top_produtos.index[0]}")
print(f"   - Receita: R$ {top_produtos.iloc[0]['Receita Total']:,.2f}")
print(f"   - Quantidade: {int(top_produtos.iloc[0]['Qtd Vendida']):,} unidades")

print(f"\n📦 PRODUTO MENOS VENDIDO: {top_produtos.index[-1]}")
print(f"   - Receita: R$ {top_produtos.iloc[-1]['Receita Total']:,.2f}")

print(f"\n🏷️ MELHOR CATEGORIA: {categorias.index[0]}")
print(f"   - Receita: R$ {categorias.iloc[0]['Receita Total']:,.2f}")
print(f"   - Participação: {categorias.iloc[0]['% Receita']:.2f}%")

print(f"\n👥 MELHOR FORNECEDOR: {top_fornecedores.index[0]}")
print(f"   - Receita: R$ {top_fornecedores.iloc[0]['Receita Total']:,.2f}")
print(f"   - Número de Produtos: {int(top_fornecedores.iloc[0]['Num Produtos'])}")

print("\n" + "=" * 80)


📋 RESUMO EXECUTIVO - PRINCIPAIS INSIGHTS

🌍 MELHOR REGIÃO: POLK
   - Receita: R$ 1,804,497.56
   - Margem: 33.32%

🌍 PIOR REGIÃO: DAVIS
   - Receita: R$ 1,107.60

🏙️ MELHOR CIDADE: DES MOINES
   - Receita: R$ 925,492.87
   - Número de Lojas: 74

🏙️ PIOR CIDADE: WYOMING
   - Receita: R$ 44.25

📦 PRODUTO MAIS VENDIDO: TITOS HANDMADE VODKA
   - Receita: R$ 378,836.72
   - Quantidade: 22,995 unidades

📦 PRODUTO MENOS VENDIDO: AMARETTO E DOLCE
   - Receita: R$ 5.01

🏷️ MELHOR CATEGORIA: CANADIAN WHISKIES
   - Receita: R$ 1,074,050.01
   - Participação: 13.33%

👥 MELHOR FORNECEDOR: DIAGEO AMERICAS
   - Receita: R$ 1,883,670.26
   - Número de Produtos: 187

